## 1. Singleton Design Pattern

The Singleton Design Pattern is focused on controlling the instantiation process of a class to ensure that only one instance exists at any given time. This is achieved by providing a static method that either creates an instance of the class if one doesn't exist, or returns the existing instance.

The Singleton pattern is useful when you want to maintain a single point of control over resources such as database connections, configuration settings, and caching mechanisms.

In [30]:
class ConfigurationManager:
    _instance = None

    def __new__(cls):
        if cls._instance is None:
            cls._instance = super(ConfigurationManager, cls).__new__(cls)
            cls._instance.config = {}
            # Initialize your configuration settings here
        return cls._instance

# Usage
config_manager1 = ConfigurationManager()
config_manager2 = ConfigurationManager()

print(config_manager1 is config_manager2) 

print(id(config_manager1) , id(config_manager2))  

True
4394763376 4394763376


### Benefits of the Singleton Pattern

1. __Single Point of Control:__ The pattern ensures that there's only one instance of the class, simplifying access to shared resources.

2. __Resource Management:__ It helps manage resources like database connections, file handles, and network sockets, preventing wasteful allocations.

3. __Memory Efficiency:__ The Singleton pattern conserves memory by maintaining a single instance instead of multiple instances.

4. __Global Access:__ Singleton instances can be accessed globally, making it easy to share data and functionality across the application.

In [31]:
import sqlite3
from threading import Lock

class SingletonMeta(type):
    """
    A thread-safe implementation of Singleton.
    """
    _instances = {}
    _lock = Lock()  # Lock object for thread safety

    def __call__(cls, *args, **kwargs):
        with cls._lock:
            if cls not in cls._instances:
                # If instance does not exist, create it
                instance = super().__call__(*args, **kwargs)
                cls._instances[cls] = instance
        return cls._instances[cls]


class DatabaseConnection(metaclass=SingletonMeta):
    """
    Singleton class for database connection.
    """
    def __init__(self, db_name):
        self._connection = None
        self._db_name = db_name

    def connect(self):
        if self._connection is None:
            self._connection = sqlite3.connect(self._db_name)
            print(f"Connected to database: {self._db_name}")
        return self._connection

    def close(self):
        if self._connection:
            self._connection.close()
            self._connection = None
            print(f"Connection to database: {self._db_name} closed.")


# Example usage
if __name__ == "__main__":
    # First instance
    db1 = DatabaseConnection("example.db")
    conn1 = db1.connect()

    # Second instance
    db2 = DatabaseConnection("example.db")
    conn2 = db2.connect()

    db3 = DatabaseConnection("example1.db")
    conn3 = db3.connect()

    # Verify both instances are the same
    print(f"Are db1 and db2 the same instance? {db1 is db2}")

    print(f"Are db1 and db3 the same instance? {db1 is db3}")

    # Close connection
    db1.close()

Connected to database: example.db
Are db1 and db2 the same instance? True
Are db1 and db3 the same instance? True
Connection to database: example.db closed.


## 2. Prototype
__Prototype__ is a creational design pattern that lets you copy existing objects without making your code dependent on their classes.

__Problem__
Say you have an object, and you want to create an exact copy of it. How would you do it? First, you have to create a new object of the same class.
Then you have to go through all the fields of the original object and copy their values over to the new object.

Nice! But there’s a catch. Not all objects can be copied that way because some of the object’s fields may be private and not visible from outside of the object itself.

There’s one more problem with the direct approach. Since you have to know the object’s class to create a duplicate, your code becomes dependent on that class.
If the extra dependency doesn’t scare you, there’s another catch. Sometimes you only know the interface that the object follows, but not its concrete class, when,
for example, a parameter in a method accepts any objects that follow some interface.

In [38]:
import copy


class Prototype:
    """
    Prototype class to clone objects.
    """
    def __init__(self, name, data):
        self.name = name
        self.data = data

    def clone(self, **attributes):
        """
        Clone the object and update attributes.
        """
        obj = copy.deepcopy(self)
        obj.__dict__.update(attributes)
        return obj

    def __str__(self):
        return f"Prototype(name={self.name}, data={self.data})"


# Example usage
if __name__ == "__main__":
    # Create an original object
    original = Prototype(name="Original", data={"key": "value"})

    # Clone the object and modify attributes
    clone1 = original.clone(name="Clone1")
    clone2 = original.clone(name="Clone2", data={"key": "new_value"})

    # Display the objects
    print(original)  # Output: Prototype(name=Original, data={'key': 'value'})
    print(clone1)    # Output: Prototype(name=Clone1, data={'key': 'value'})
    print(clone2)    # Output: Prototype(name=Clone2, data={'key': 'new_value'})


Prototype(name=Original, data={'key': 'value'})
Prototype(name=Clone1, data={'key': 'value'})
Prototype(name=Clone2, data={'key': 'new_value'})


In [33]:
import copy


class Prototype:
    """
    Prototype class that supports cloning of objects.
    """
    def __init__(self):
        self._objects = {}

    def register_object(self, key, obj):
        """Register an object with a key."""
        self._objects[key] = obj

    def unregister_object(self, key):
        """Unregister an object by its key."""
        if key in self._objects:
            del self._objects[key]

    def clone(self, key, **attributes):
        """
        Clone a registered object and optionally update its attributes.
        """
        obj = copy.deepcopy(self._objects.get(key))
        if not obj:
            raise ValueError(f"No object found for key: {key}")
        obj.__dict__.update(attributes)
        return obj


# Example class to demonstrate the Prototype pattern
class Car:
    def __init__(self, make, model, color):
        self.make = make
        self.model = model
        self.color = color

    def __str__(self):
        return f"Car(make={self.make}, model={self.model}, color={self.color})"

#### Explanation:

__Prototype Class:__ 
* Acts as a manager to register and clone objects.
* Stores prototypes in a dictionary and allows cloning them using the clone method.

__Car Class:__

* An example class whose instances can be used as prototypes.
* Contains attributes like make, model, and color.

__Deep Copy:__
* The copy.deepcopy function ensures that cloned objects are completely independent of the original prototype.

__Attributes Update:__
* The clone method allows passing updated attributes during the cloning process.

In [34]:
# Example usage
if __name__ == "__main__":
    # Create the prototype manager
    prototype_manager = Prototype()

    # Create a base prototype
    car1 = Car("Toyota", "Corolla", "White")
    prototype_manager.register_object("basic_car", car1)

    # Clone the prototype and modify attributes
    car2 = prototype_manager.clone("basic_car", color="Red")
    car3 = prototype_manager.clone("basic_car", model="Camry", color="Blue")

    # Display cloned objects
    print(car1)  # Original prototype
    print(car2)  # Cloned with modified color
    print(car3)  # Cloned with modified model and color

Car(make=Toyota, model=Corolla, color=White)
Car(make=Toyota, model=Corolla, color=Red)
Car(make=Toyota, model=Camry, color=Blue)


__Use Cases:__
1. When object creation is expensive, and you want to avoid the cost by cloning existing objects.
2. When you need variations of an object but want to base them on a pre-configured prototype.
3. Example scenarios: Game characters, document templates, or configuration objects.

#### Real Use Case: game development scenario
A real-world use case for the Prototype Design Pattern is in a game development scenario where you have multiple types of objects (e.g., enemies, weapons, or power-ups) that need to be created frequently. Instead of instantiating these objects from scratch, you can clone pre-configured prototypes.

In [47]:
import copy


class GameCharacter:
    """
    A class representing a game character.
    """
    def __init__(self, name, health, attack, defense, abilities=None):
        self.name = name
        self.health = health
        self.attack = attack
        self.defense = defense
        self.__abilities = abilities if abilities else []

    def clone(self, **attributes):
        """
        Clone the character and update attributes.
        """
        obj = copy.deepcopy(self)
        obj.__dict__.update(attributes)
        return obj

    def __str__(self):
        return (f"GameCharacter(name={self.name}, health={self.health}, "
                f"attack={self.attack}, defense={self.defense}, abilities={self.__abilities})")


# Example usage
if __name__ == "__main__":
    # Create a prototype character
    orc_prototype = GameCharacter(name="Orc", health=100, attack=15, defense=10, abilities=["Smash", "Roar"])

    # Clone the prototype for different scenarios
    elite_orc = orc_prototype.clone(name="Elite Orc", health=150, attack=25, abilities=["Smash", "Roar", "Charge"])
    boss_orc = orc_prototype.clone(name="Boss Orc", health=300, attack=50, defense=20)

    # Display the characters
    print(orc_prototype)  # Prototype
    print(elite_orc)      # Cloned with modified attributes
    print(boss_orc)       # Cloned with different modifications


GameCharacter(name=Orc, health=100, attack=15, defense=10, abilities=['Smash', 'Roar'])
GameCharacter(name=Elite Orc, health=150, attack=25, defense=10, abilities=['Smash', 'Roar'])
GameCharacter(name=Boss Orc, health=300, attack=50, defense=20, abilities=['Smash', 'Roar'])


## 3. Factory Method
Factory Method is a creational design pattern that provides an interface for creating objects in a superclass, but allows subclasses to alter the type of objects that will be created.

__Problem__

Imagine that you’re creating a logistics management application. The first version of your app can only handle transportation by __trucks__, so the bulk of your code lives inside the Truck class.


After a while, your app becomes pretty popular. Each day you receive dozens of requests from sea transportation companies to incorporate __sea__ logistics into the app.


Great news, right? But how about the code? At present, most of your code is coupled to the __Truck class__. Adding Ships into the app would require making changes to the entire codebase. Moreover, if later you decide to add another type of transportation to the app, you will probably need to make all of these changes again.

As a result, you will end up with pretty nasty code, riddled with conditionals that switch the app’s behavior depending on the class of transportation objects.




In [50]:
from abc import ABC, abstractmethod

# Abstract Product
class Vehicle(ABC):
    @abstractmethod
    def create(self):
        pass

# Concrete Products
class Car(Vehicle):
    def create(self):
        return "Car created"

class Bike(Vehicle):
    def create(self):
        return "Bike created"

# Creator (Factory Method)
class VehicleFactory(ABC):
    @abstractmethod
    def get_vehicle(self):
        pass

# Concrete Factories
class CarFactory(VehicleFactory):
    def get_vehicle(self):
        return Car()

class BikeFactory(VehicleFactory):
    def get_vehicle(self):
        return Bike()

# Client Code
if __name__ == "__main__":
    # Create a car factory
    car_factory = CarFactory()
    car = car_factory.get_vehicle()
    print(car.create())  # Output: Car created

    # Create a bike factory
    bike_factory = BikeFactory()
    bike = bike_factory.get_vehicle()
    print(bike.create())  # Output: Bike created


Car created
Bike created


### Explanation:
1. Abstract Product (Vehicle):
    * Defines the interface for the objects created by the factory.

2. Concrete Products (Car, Bike):
    * Implements the Vehicle interface. Each subclass represents a specific type of object.

3. Creator (VehicleFactory):
    * Declares the factory method (__get_vehicle__) for creating objects.
    * Subclasses implement this method to specify the type of object to be instantiated.

4. Concrete Factories (CarFactory, BikeFactory):
    * Implements the factory method and returns specific objects (e.g., Car or Bike).

5. Client Code:
    * Uses the factory method (__get_vehicle__) to get an instance of a product without knowing the exact class of the object.

__Real-Life Example: Notification System__

Imagine a notification system that sends messages through different channels (e.g., Email, SMS, or Push Notification). The Factory Method can help create the appropriate notification sender without the client code needing to know the details.

In [51]:
from abc import ABC, abstractmethod

# Abstract Product
class Notification(ABC):
    @abstractmethod
    def send(self, message: str):
        pass


# Concrete Products
class EmailNotification(Notification):
    def send(self, message: str):
        return f"Email sent: {message}"


class SMSNotification(Notification):
    def send(self, message: str):
        return f"SMS sent: {message}"


class PushNotification(Notification):
    def send(self, message: str):
        return f"Push Notification sent: {message}"


# Creator (Factory Method)
class NotificationFactory(ABC):
    @abstractmethod
    def create_notification(self) -> Notification:
        pass


# Concrete Factories
class EmailNotificationFactory(NotificationFactory):
    def create_notification(self) -> Notification:
        return EmailNotification()


class SMSNotificationFactory(NotificationFactory):
    def create_notification(self) -> Notification:
        return SMSNotification()


class PushNotificationFactory(NotificationFactory):
    def create_notification(self) -> Notification:
        return PushNotification()


# Client Code
def send_notification(factory: NotificationFactory, message: str):
    notification = factory.create_notification()
    print(notification.send(message))


if __name__ == "__main__":
    # Sending an Email
    email_factory = EmailNotificationFactory()
    send_notification(email_factory, "Welcome to our service!")

    # Sending an SMS
    sms_factory = SMSNotificationFactory()
    send_notification(sms_factory, "Your OTP is 123456")

    # Sending a Push Notification
    push_factory = PushNotificationFactory()
    send_notification(push_factory, "You have a new friend request")


Email sent: Welcome to our service!
SMS sent: Your OTP is 123456
Push Notification sent: You have a new friend request


## 4.Builder
Builder is a creational design pattern that lets you construct complex objects step by step. The pattern allows you to produce different types and representations of an object using the same construction code

In [55]:
class House:
    """
    The Product class represents the complex object being built.
    """
    def __init__(self):
        self.walls = None
        self.roof = None
        self.floor = None
        self.windows = None
        self.doors = None

    def __str__(self):
        return f"House with {self.walls} walls, {self.roof} roof, {self.floor} floor, " \
               f"{self.windows} windows, and {self.doors} doors."


class HouseBuilder:
    """
    The Builder class provides methods to construct different parts of the product.
    """
    def __init__(self):
        self.house = House()

    def build_walls(self, walls):
        self.house.walls = walls
        return self

    def build_roof(self, roof):
        self.house.roof = roof
        return self

    def build_floor(self, floor):
        self.house.floor = floor
        return self

    def build_windows(self, windows):
        self.house.windows = windows
        return self

    def build_doors(self, doors):
        self.house.doors = doors
        return self

    def get_result(self):
        """
        Returns the final product.
        """
        return self.house


# Example usage
if __name__ == "__main__":
    # Create a builder
    builder = HouseBuilder()

    # Build a house step by step
    house = (builder
             .build_walls("brick")
             .build_roof("tile")
             .build_floor("wooden")
             .build_windows(4)
             .build_doors(2)
             .get_result())

    print(house)  # Output: House with brick walls, tile roof, wooden floor, 4 windows, and 2 doors.


House with brick walls, tile roof, wooden floor, 4 windows, and 2 doors.
